# Refinement in the windows 



In [7]:
import os
COCO_API_DIR = 'D:/Datasets/COCO/cocoapi-master/PythonAPI'
COCO_ROOT = 'D:/Datasets/COCO/'
COCO_SET = 'train2017'
import sys
import threading
from threading import Thread
import time
sys.path.append('../frontend')
sys.path.append('../backend')
sys.path.append(COCO_API_DIR)

import cv2
import datetime
import pickle
import json
import time
import glob
import timeit
import skimage.io as io
from scipy.ndimage.filters import gaussian_filter
from keras.layers import *
import keras
from keras.models import *
from keras import losses
from keras.callbacks import TensorBoard
from keras.optimizers import *
from IPython.display import clear_output
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import random
import datasets
import composition

In [2]:
annFile='{}/annotations/instances_{}.json'.format(COCO_ROOT,COCO_SET)
annFileKPS = '{}/annotations/person_keypoints_{}.json'.format(COCO_ROOT,COCO_SET)
catNms = ['person']
coco=COCO(annFile)
coco_kps=COCO(annFileKPS)  
catIds = coco.getCatIds(catNms)
imgIds = coco.getImgIds(catIds=catIds )

loading annotations into memory...
Done (t=17.79s)
creating index...
index created!
loading annotations into memory...
Done (t=9.84s)
creating index...
index created!


In [3]:
def get_real_examples(list_out, coco, coco_kps, coco_root, imgIds, catIds, params, batch_size):
    print('Starting the real examples thread')
    # GET REFERENCE, NATURAL LOOKING EXAMPLES
    for i in range(0, batch_size):
        IM = composition.get_image_suitable_for_blending_with_meta(coco, coco_kps, coco_root, imgIds, catIds, longer_edge_size=params['longer_edge_size'], scale_range=params['scale_range_A'], shift_range=params['shift_range_A'], rot_range=params['rot_range_A'], allow_overlaps=params['allow_overlaps_in_A'])
        list_out.append(IM)
    print('Real examples done')

def get_fake_examples(list_out, coco, coco_kps, coco_root, imgIds, catIds, params, batch_size):
    print('Starting the fake examples thread')
    # GET GENERATED EXAMPLES
    for i in range(0, batch_size):
        while True:
            data_train = composition.get_composition(coco, coco_kps,coco_root, imgIds, catIds, params=params)
            if data_train is not None:
                list_out.append(data_train)
                break
    print('Fake examples done')
    

def get_training_examples(coco, coco_kps, coco_root, imgIds, catIds, composition_params, batch_size):
    
    list_real = list()
    list_fake = list()
    thread_real = Thread(target=get_real_examples, args=(list_real, coco, coco_kps, coco_root, imgIds, catIds, composition_params, batch_size))
    thread_fake = Thread(target=get_fake_examples, args=(list_fake, coco, coco_kps, coco_root, imgIds, catIds, composition_params, batch_size))
    
    thread_real.start()
    thread_fake.start()

    thread_real.join()
    thread_fake.join()
    return list_real, list_fake

# Get initial parameters
batch_size = 2
train_data = get_training_examples(coco, coco_kps, COCO_ROOT, imgIds, catIds, composition.default_composition_params, batch_size)




Starting the real examples threadStarting the fake examples thread

Real examples done
Fake examples done


In [14]:
# 
vgg_front = keras.applications.vgg19.VGG19(
    include_top=False, 
    weights='imagenet', 
    input_tensor=None, 
    input_shape=(255,255,3), 
    pooling=None)
vgg_front.summary()

vgg_input = vgg_front.inputs[0]
x = vgg_front(vgg_input)




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 255, 255, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 255, 255, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 255, 255, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 127, 127, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 127, 127, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 127, 127, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 63, 63, 128)       0         
__________

TypeError: 'Tensor' object is not callable

In [ ]:
# LET'S BUILD A MODEL NOW
def stage_block(features, 
                prev_block=None, 
                stage_name='KD', 
                num_kernels_per_step=128, 
                num_kernels_out=128,
                num_convs=7, 
                padding='same',
               prefix=''):
    
    if prev_block is not None:
        x = concatenate([features, prev_block],3)
    else:
        x = features
    for i in range(0, num_convs-2):
        x = Conv2D(num_kernels_per_step, 
                   (7,7), 
                   strides=(1,1), 
                   padding=padding, 
                   kernel_initializer='he_uniform', 
                   name='{}Mconv{}_stage{}'.format(prefix,i, stage_name), activation='relu')(x)
        
    x = Conv2D(num_kernels_per_step, (1,1), strides=(1,1), padding='valid', kernel_initializer='he_uniform', name='{}Mconv{}_stage{}'.format(prefix,num_convs-1,stage_name), activation='relu')(x)
    x = Conv2D(num_kernels_out, (1,1), strides=(1,1), padding='valid',kernel_initializer='he_uniform', name='{}Mconv{}_stage{}'.format(prefix,num_convs,stage_name))(x)
    return x


# DISCRIMINATOR - REAL VS FAKE
dis_input = Input((256, 256, 3))
x = dis_input
for i in range(0,7):
    x = stage_block(x, num_convs=3, prefix='dsx_{}_'.format(i))
    x = MaxPool2D((2,2))(x)
x = Flatten()(x)
x = Dense(1, kernel_initializer='he_uniform')(x)
dis_model = Model(dis_input, x)

dis_model.trainable = False

# REFINERS and FULL MODELS
ref_models = []
full_models = []
num_refiners = 9
num_kernels_per_step = 32
ref_input = Input((256,256,11)) # 3 + 3 +3 + 2, A, B, C, mask A, mask B 
for i in range(0, num_refiners):
    x = Conv2D(num_kernels_per_step, (3,3), strides=(1,1),  kernel_initializer='he_uniform', padding='same')(ref_input)
    x = LeakyReLU(0.1)(x)
    for d in range(0, i):
        x = Conv2D(num_kernels_per_step, (3,3), strides=(1,1),  kernel_initializer='he_uniform', padding='same')(x)
        x = LeakyReLU(0.1)(x)
        x = Conv2D(num_kernels_per_step, (3,3), strides=(1,1),  kernel_initializer='he_uniform', padding='same')(x)
        x = LeakyReLU(0.1)(x)
    x = Conv2D(3, (3,3), strides=(1,1), padding='same',  kernel_initializer='he_uniform')(x)
    ref_output = x
    ref_model = Model(ref_input, ref_output)
    ref_models.append(ref_model)
    
    full_output = dis_model( ref_output )
    full_model = Model(ref_input, full_output)
    full_models.append(full_model)




In [ ]:
print('FULL MODELS\n-----------------------------')
dis_model.trainable = False
for i,full_model in enumerate(full_models):
    print('FULL MODEL {}'.format(i))
    full_opt = optimizers.Adam(lr=0.00005)
    full_model.compile(full_opt, 'mean_absolute_error')
    full_model.summary()

print('DISCRIMINATOR')
dis_model.trainable = True
dis_opt = optimizers.Adam(lr=0.00005)
dis_model.compile(dis_opt, 'mean_absolute_error')
dis_model.summary()

print('REFINERS\n------------------------------')
for i,ref_model in enumerate(ref_models):
    print('REFINER {}'.format(i))
    ref_opt = optimizers.Adam(lr=0.00001)
    ref_model.compile(ref_opt, 'mean_absolute_error')
    ref_model.summary()

In [ ]:

def visualize_progress(progress):
    
    if len(progress) < 3:
        return
    
    labels = []
    x = np.array([ progress[i][0] for i in range(0, len(progress)) ])
    
    gen_dis_loss = np.array([ progress[i][1] for i in range(0, len(progress)) ])
    labels.append('Gen Dis')
    
    dis_losses = np.array([ progress[i][2] for i in range(0, len(progress)) ])
    for i in range(0,dis_losses.shape[1]):
        labels.append('Dis {}'.format(i))

    ref_losses = np.array([ progress[i][3] for i in range(0, len(progress)) ])
    for i in range(0,ref_losses.shape[1]):
        labels.append('Ref {}'.format(i))

        
    full_losses = np.array([ progress[i][4] for i in range(0, len(progress)) ])
    for i in range(0,full_losses.shape[1]):
        labels.append('Full {}'.format(i))

        
    # TRIMMING
    histlen = 100
    x = x[ np.max([0, x.shape[0]-histlen]):, ]
    gen_dis_loss = gen_dis_loss[ np.max([0, gen_dis_loss.shape[0]-histlen]):, ]
    dis_losses = dis_losses[ np.max([0, dis_losses.shape[0]-histlen]):, ]        
    ref_losses = ref_losses[ np.max([0, ref_losses.shape[0]-histlen]):, ]        
    full_losses = full_losses[ np.max([0, full_losses.shape[0]-histlen]):, ]        
        
    print('Last dis losses: {}'.format(dis_losses[-1,:]))
    print('Last full losses: {}'.format(full_losses[-1,:]))
        
    plt.figure(figsize=(12,5))
    plt.plot(x ,gen_dis_loss)
    for i in range(0, dis_losses.shape[1]):
        plt.plot(x, dis_losses[:,i])
    for i in range(0, full_losses.shape[1]):
        plt.plot(x, full_losses[:,i])
    plt.legend(labels)
    plt.grid()
    plt.show()

def show_ref_viz(ref_viz):
    img = np.zeros((256, 256*len(ref_viz), 3))
    img[:, :256,:] = ref_viz[0][:,:,0:3]
    for i in range(1, len(ref_viz)):
        img[:, i*256:(i+1)*256, :] = np.clip(ref_viz[i], 0, 255)
        
    plt.figure(figsize=(20,7))
    plt.imshow(img.astype(np.uint8))
    plt.show()
    return img.astype(np.uint8)



In [ ]:
it = 0
progress = []
composition_params = composition.default_composition_params
composition_params['longer_edge_size'] = 256

In [ ]:



while True:

    # Set up the threads to work in the background while running GPU tasks
    list_real = list()
    list_fake = list()
    thread_real = Thread(target=get_real_examples, args=(list_real, coco, coco_kps, COCO_ROOT, imgIds, catIds, composition.default_composition_params, batch_size))
    thread_fake = Thread(target=get_fake_examples, args=(list_fake, coco, coco_kps, COCO_ROOT, imgIds, catIds, composition.default_composition_params, batch_size))
    
    thread_real.start()
    thread_fake.start()
    
    # ONLY IF WE HAVE THE EXAMPLES, THUS BEING ITERATION NUMBER > 0
    if it > 0:
        # TRAIN DISCRIMINATOR FIRST ON REAL VS STANDARD COMPOSITION
        print('General discriminator training')
        dis_model.trainable = True
        gen_dis_loss_cur = dis_model.train_on_batch(dis_x, dis_y)

        # TRAIN THE FULL MODELS
        dis_losses = [0]*num_refiners
        ref_losses = [0]*num_refiners
        full_losses = [0]*num_refiners
        for i in range(0, num_refiners):
            print('Discriminator training for predictions from ref {}'.format(i))
            dis_model.trainable = True
            # Get prediction from particular refiner
            dis_x_fake_pred = ref_models[i].predict( ref_x[batch_size//2:,] )
            
            dis_loss_cur = dis_model.train_on_batch(np.concatenate([ dis_x_real,dis_x_fake_pred ],0), dis_y)
            dis_losses[i] = dis_loss_cur
            
            # Train in AE task
            if 0:
                print('Refiner {} training'.format(i))
                ref_batch_x = ref_x
                ref_batch_y = ref_x[:,:,:,0:3]
                ref_loss_cur = ref_models[i].train_on_batch(ref_batch_x, ref_batch_y)
                ref_losses[i] = ref_loss_cur
            ref_losses[i] = 0 
            
            # Train the whole model next
            print('Full {} training'.format(i))
            dis_model.trainable = False
            full_batch_x = ref_x
            full_batch_y = np.ones((batch_size, 1))
            full_loss_cur = full_models[i].train_on_batch(full_batch_x, full_batch_y)
            full_losses[i] = full_loss_cur
        print('Dis losses: {}'.format(dis_losses))
        print('Full losees: {}'.format(full_losses))
        
        progress.append([ it, gen_dis_loss_cur, dis_losses, ref_losses, full_losses ])
        
    it += 1
    
    # Wait for the threads to stop and store new training data
    thread_real.join()
    thread_fake.join()
    
    
    
    # PREP THE TRAINING DATA
    ref_x = np.array([ np.concatenate([ list_fake[i]['image_scaled_and_padded'],
                                    list_fake[i]['A']['image_scaled_and_padded'],
                                   list_fake[i]['B']['image_scaled_and_padded'],
                                   np.expand_dims(list_fake[i]['A']['composite_mask'],2),
                                     np.expand_dims(list_fake[i]['inserted_mask'], 2)
                                  ], 2) for i in range(0,batch_size) ])

    dis_x_real = np.array([ list_real[i]['image_scaled_and_padded'] for i in range(0, batch_size//2) ])
    dis_x_fake = np.array([ list_fake[i]['image_scaled_and_padded'] for i in range(0, batch_size//2) ])
    dis_x = np.concatenate([ dis_x_real, dis_x_fake ], 0)
    dis_y = np.zeros((batch_size,1))
    dis_y[0:batch_size//2,0] = 1
    
    
    if it % 2 == 0 and it > 2:
        
        ref_viz = [ ref_x[0] ]
        for i in range(0, num_refiners):
            dis_x_fake_pred = ref_models[i].predict( ref_x[0:1] )
            ref_viz.append( dis_x_fake_pred[0] )
            
        clear_output()
        img = show_ref_viz(ref_viz)
        visualize_progress(progress)
        try:
            cv2.imwrite('current_progress.png', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        except:
            print('For some reason unable to write a file...')
    